#STEP -1 : Add the Required Libraries

In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [ ]:
!pip install https://github.com/PyThaiNLP/pythainlp/archive/dev.zip
!pip install https://github.com/PyThaiNLP/spelling_check/archive/master.zip

     \ 11.7 MB 9.5 MB/s
  Created wheel for pythainlp: filename=pythainlp-3.0.5-py3-none-any.whl size=11506829 sha256=f5690a2455d17e004f194f9d98fc34a4f9992a19c82848520dd22b4203b38cd8
  Stored in directory: /tmp/pip-ephem-wheel-cache-qo36373l/wheels/87/69/e8/234fe11c5f2ec389a780229b2a1619e17b8ad38cd45eaf20f7
Successfully built pythainlp
     \ 313 kB 3.7 MB/s
     |████████████████████████████████| 743 kB 32.2 MB/s 
  Created wheel for pythaispell: filename=pythaispell-0.1-py3-none-any.whl size=235096 sha256=5d0ccc1e16e57df7837b18e5dda03b94fd11e063645d32f68dabd124a3912c23
  Stored in directory: /tmp/pip-ephem-wheel-cache-w_k5aalr/wheels/d6/3b/61/8f95697c7696e4d3e0ad9d769b59cfc05c2ca36a8a35e51ab3
Successfully built pythaispell


#STEP -2: Set random seed

In [ ]:
np.random.seed(500)

#STEP -3: Add the Corpus

In [ ]:
Corpus = pd.read_excel(r"/content/drive/MyDrive/3 2/tweet-bullying-dataset.xlsx")

In [ ]:
Corpus

,tweet,date,reply,retweet,like,bully
0,เนี่ยยยบอกแล้วคนเดี๋ยวนี้ไม่รู้ภาษาเยอะมากไม่ม...,2022-03-01T07:59:12.000Z,0,NaN,0,yes
1,อายเด็กอ่ะอยู่ปมีแฟนกันแล้วดูกุดิปัญญาอ่อนไปวั...,2022-03-09T07:57:04.000Z,0,NaN,0,yes
2,ค่ะเจ๊เลิกเงี่ยนกับกะเทยป้าแก่เล่นกล้ามก่อน,2022-02-28T13:05:46.000Z,0,3,0,yes
3,เจอผญคนนี้อีกละแต่งหน้าเหมือนไปเล่นงิ้วมากไม่ไ...,2015-07-17T11:45:06.000Z,0,NaN,0,yes
4,สสวิปริตเป็นพรรคที่มีสสบ้าบอบ้าตัณหาทางการผิดเ...,2022-02-11T01:39:03.000Z,0,0,0,yes
...,...,...,...,...,...,...
1195,ทักหน่อยสิ,2022-03-01T04:27:57.000Z,1,NaN,0,no
1196,แทคได้พูดแทนกูไปหมดล้ะ,2022-03-01T00:16:08.000Z,0,289,0,no
1197,เออว่ะมันเป็นรายการวาไรตี้เหมือนกันนี่หว่าโรเซ...,2022-01-27T03:38:25.000Z,0,1203,0,no
1198,ใครมีเลขเด็ดเลขดังมาแชร์กันบ้างน่ะ,2022-03-01T02:06:24.000Z,0,NaN,0,no


#STEP -4: Data pre-processing

In [ ]:
!pip install iteration_utilities
from pythainlp.tokenize import word_tokenize
from iteration_utilities import duplicates
from pythainlp.util import normalize
import re
import pythaispell
from pythainlp.corpus import thai_stopwords

def word_tokenizer(text):
  # text = 'นิ้วเบียดเก่งพิมพ์ตกเก่งงงงงง'
  text  = pythaispell.spell(text ,autocorrect=True)
  text = normalize(text) #สระะน้ำ-->> สระน้ำ
  list_word = word_tokenize(text, engine="longest")
  # print(list_word)
  stopwords = list(thai_stopwords())
  list_word_not_stopwords = [i for i in list_word if i not in stopwords]
  print(list_word_not_stopwords)
  return list_word_not_stopwords


def remove_one_word(list_word_not_stopwords):
  for word in list_word_not_stopwords:
    # print(word)
    if not re.findall(r"[ฯะัาำิีึืฺุูเแโใไๅๆ็่้๊๋์]", word) and len(re.findall(r"[ก-ฮ]{1}", word)) <2:
      list_word_not_stopwords.remove(word)
      print('remove :' ,word)
  return 

def remove_emotional_word(list_word_not_stopwords):
    l = []
    for word in list_word_not_stopwords:
      if len(re.findall(r"[ฯะัาำิีึืฺุูเแโใไๅๆ็่้๊๋์]", word))<1:
        l.append(word)
    # print(l)
    for w in l:
      c = re.findall(r"[ก-ฮ]{1}", w)
      # print(c, len(c))
      if len(list(duplicates(c))) == len(c)-1:
        print(list(duplicates(c)))
        print(len(list(duplicates(c))))
        print('remove :' ,word)
        try:
          list_word_not_stopwords.remove(w)
        except:
          pass
    return 

     |████████████████████████████████| 283 kB 24.0 MB/s 


In [ ]:
!pip install deepcut
# list_text = ["ตามมาดูสิไอ้พวกผิดเพศ", "เร็วมากถึงนาทีไหม", "ทีมงานเซ็ตนี้หน้าเหี้ยแล้วยังทำตัวเหี้ยๆอีกนะครับ", "มีเงินซื้อสตาบัคแต่ไม่มีเงินซื้อมารยาท", "วิวดีจริงงับบบงื้ออออ"]
list_text = ["เจอผญคนนี้อีกละแต่งหน้าเหมือนไปเล่นงิ้วมากไม่ได้พูดเกินไปนะคือหน้าขาวมากขาวแต่หน้าชมพูมาตั้งแต่ตาถึงแก้มปากก็แดงกรี๊ดดดด", "อันนี้พ่อแม่เคยสอนมั้ยว่าจะติใครควรใช้คำดีๆหรือไม่มีพ่อแม่แต่ก็นะต่อให้มีพ่อแม่สอนแกก็คงเหี้ยเหมือนเดิมนั่นแหละแล้วก็นะเง่นก็ไปชตอไม่ก็ซื้อกินอย่าไปพรากผู้เยาว์เลยน้องเขามีพ่อแม่ไม่ควรมาเจอเสนียดจัญไรแบบคนอย่างแกด้วยซ้ำ", "เนี่ยยยบอกแล้วคนเดี๋ยวนี้ไม่รู้ภาษาเยอะมากไม่มีมารยาทก็เยอะมาอยู่กรุงเทพเจอคนแบบนี้เยอะมากส่ายหัวแล้วส่ายหัวอีกที่บ้านไม่มีใครสอนเรื่องมารยาทหรือยังไงหรือนิสัยเป็นแบบนี้แก้ไม่หาย", "มีเงินซื้อสตาบัคแต่ไม่มีเงินซื้อมารยาท"]
for tweet in list_text:
  list_word = word_tokenize(tweet, engine="deepcut")
  print(list_word)
  # list_word_not_stopwords = word_tokenizer(tweet)
  # remove_one_word(list_word_not_stopwords)
  # remove_emotional_word(list_word_not_stopwords)
  # tokenize_list.append(list_word_not_stopwords)
  # print(list_word_not_stopwords)

['เจอ', 'ผญ', 'คน', 'นี้', 'อีก', 'ละ', 'แต่ง', 'หน้า', 'เหมือน', 'ไป', 'เล่น', 'งิ้ว', 'มาก', 'ไม่', 'ได้', 'พูด', 'เกิน', 'ไป', 'นะ', 'คือ', 'หน้า', 'ขาว', 'มาก', 'ขาว', 'แต่', 'หน้า', 'ชมพู', 'มา', 'ตั้งแต่', 'ตา', 'ถึง', 'แก้ม', 'ปาก', 'ก็', 'แดง', 'กรี๊ดดดด']
['อัน', 'นี้', 'พ่อแม่', 'เคย', 'สอน', 'มั้ย', 'ว่า', 'จะ', 'ติ', 'ใคร', 'ควร', 'ใช้', 'คำ', 'ดี', 'ๆ', 'หรือ', 'ไม่', 'มี', 'พ่อแม่', 'แต่', 'ก็', 'นะ', 'ต่อ', 'ให้', 'มี', 'พ่อแม่', 'สอน', 'แก', 'ก็', 'คง', 'เหี้ย', 'เหมือน', 'เดิม', 'นั่น', 'แหละ', 'แล้ว', 'ก็', 'นะ', 'เง่น', 'ก็', 'ไป', 'ชตอ', 'ไม่', 'ก็', 'ซื้อ', 'กิน', 'อย่า', 'ไป', 'พราก', 'ผู้', 'เยาว์', 'เลย', 'น้อง', 'เขา', 'มี', 'พ่อแม่', 'ไม่', 'ควร', 'มา', 'เจอ', 'เสนียดจัญไร', 'แบบ', 'คน', 'อย่าง', 'แก', 'ด้วยซ้ำ']
['เนี่ยยย', 'บอก', 'แล้ว', 'คน', 'เดี๋ยวนี้', 'ไม่', 'รู้', 'ภาษา', 'เยอะ', 'มาก', 'ไม่', 'มี', 'มารยาท', 'ก็', 'เยอะ', 'มา', 'อยู่', 'กรุงเทพ', 'เจอ', 'คน', 'แบบ', 'นี้', 'เยอะ', 'มาก', 'ส่าย', 'หัว', 'แล้ว', 'ส่าย', 'หัว', 'อีก', 'ที่', 'บ้าน', 'ไม่

In [ ]:
อัน|นี้|พ่อแม่|เคย|สอน|มั้ย|ว่า|จะ|ติ|ใคร|ควร|ใช้|คำ|ดี|ๆ|หรือ|ไม่|มี|พ่อแม่|แต่|ก็|นะ|ต่อ|ให้|มี|พ่อแม่|สอน|แก|ก็|คง|เหี้ย|เหมือน|เดิม|นั่น|แหละ|แล้ว|ก็|นะ|เง่น|ก็|ไป|ชตอ|ไม่|ก็|ซื้อ|กิน|อย่า|ไป|พราก|ผู้|เยาว์|เลย|น้อง|เขา|มี|พ่อแม่|ไม่|ควร|มา|เจอ|เสนียดจัญไร|แบบ|คน|อย่าง|แก|ด้วยซ้ำ

In [ ]:
from pythainlp import tokenize
# Corpus_test = Corpus[:5]
tokenize_list = []
for tweet in Corpus['tweet']:
  list_word_not_stopwords = word_tokenizer(tweet)
  remove_one_word(list_word_not_stopwords)
  remove_emotional_word(list_word_not_stopwords)
  tokenize_list.append(list_word_not_stopwords)
  print(list_word_not_stopwords)

['คน', 'เดี๋ยวนี้', 'รู้', 'ภาษา', 'มารยาท', 'กรุงเทพ', 'เจอ', 'คน', 'แบบนี้', 'ส่าย', 'หัว', 'ส่าย', 'หัว', 'บ้าน', 'สอน', 'เรื่อง', 'มารยาท', 'นิสัย', 'แบบนี้', 'แก้', 'หาย']
['อาย', 'เด็ก', 'อ่ะ', 'แฟน', 'ดู', 'กุ', 'ดิ', 'ปัญญาอ่อน', 'วันๆ', 'อยู่เลย']
['เจ๊', 'เลิก', 'เงี่ยน', 'กะเทย', 'ป้า', 'เล่นกล้าม']
['เจอ', 'ผญ', 'คน', 'แต่งหน้า', 'เหมือน', 'เล่น', 'งิ้ว', 'เกินไป', 'หน้า', 'ขาว', 'ขาว', 'หน้า', 'ชมพู', 'ตา', 'แก้ม', 'ปาก', 'แดง', 'กรี๊ด']
['วิปริต', 'บ้าบอ', 'บ้า', 'ตัณหา', 'ทางการ', 'ผิดเพศ', 'ขยะ']
['เหมือน', 'ไอ้', 'ยัด', 'ดาก', 'นั้นแหละ', 'ตุ๊ด', 'อัป', 'รี', 'สวะ', 'ขยะสังคม', 'ไอ้', 'วิปริต', 'ผิดเพศ', 'ขออภัย', 'หยาบ', 'ทน', 'เดรัจฉาน', 'ตัว', 'เอดส์', 'ตาย', 'โลก', 'สักที']
['อันนี้', 'พ่อแม่', 'สอน', 'ติ', 'ดีๆ', 'พ่อแม่', 'ต่อให้', 'พ่อแม่', 'สอน', 'แก็ง', 'เหี้ย', 'เหมือนเดิม', 'แล้วก็', 'เล่น', 'ตอ', 'ซื้อ', 'กิน', 'อย่า', 'พรากผู้เยาว์', 'น้อง', 'พ่อแม่', 'เจอ', 'เสนียดจัญไร', 'คน', 'ด้วยซ้ำ']
['เจอ', 'เหมือนกัน', 'อี', 'บุคคล', 'น่ารำคาญ', 'ตอนแรก', 'อธิบาย',

In [ ]:
Corpus['text_final'] = tokenize_list
Corpus

,tweet,date,reply,retweet,like,bully,text_final
0,เนี่ยยยบอกแล้วคนเดี๋ยวนี้ไม่รู้ภาษาเยอะมากไม่ม...,2022-03-01T07:59:12.000Z,0,NaN,0,yes,"[คน, เดี๋ยวนี้, รู้, ภาษา, มารยาท, กรุงเทพ, เจ..."
1,อายเด็กอ่ะอยู่ปมีแฟนกันแล้วดูกุดิปัญญาอ่อนไปวั...,2022-03-09T07:57:04.000Z,0,NaN,0,yes,"[อาย, เด็ก, อ่ะ, แฟน, ดู, กุ, ดิ, ปัญญาอ่อน, ว..."
2,ค่ะเจ๊เลิกเงี่ยนกับกะเทยป้าแก่เล่นกล้ามก่อน,2022-02-28T13:05:46.000Z,0,3,0,yes,"[เจ๊, เลิก, เงี่ยน, กะเทย, ป้า, เล่นกล้าม]"
3,เจอผญคนนี้อีกละแต่งหน้าเหมือนไปเล่นงิ้วมากไม่ไ...,2015-07-17T11:45:06.000Z,0,NaN,0,yes,"[เจอ, ผญ, คน, แต่งหน้า, เหมือน, เล่น, งิ้ว, เก..."
4,สสวิปริตเป็นพรรคที่มีสสบ้าบอบ้าตัณหาทางการผิดเ...,2022-02-11T01:39:03.000Z,0,0,0,yes,"[วิปริต, บ้าบอ, บ้า, ตัณหา, ทางการ, ผิดเพศ, ขยะ]"
...,...,...,...,...,...,...,...
1195,ทักหน่อยสิ,2022-03-01T04:27:57.000Z,1,NaN,0,no,"[ทัก, สิ]"
1196,แทคได้พูดแทนกูไปหมดล้ะ,2022-03-01T00:16:08.000Z,0,289,0,no,"[แทค, แทน]"
1197,เออว่ะมันเป็นรายการวาไรตี้เหมือนกันนี่หว่าโรเซ...,2022-01-27T03:38:25.000Z,0,1203,0,no,"[เออ, ว่ะ, เป็นราย, วาไรตี้, เหมือนกัน, หว่า, ..."
1198,ใครมีเลขเด็ดเลขดังมาแชร์กันบ้างน่ะ,2022-03-01T02:06:24.000Z,0,NaN,0,no,"[เลขเด็ด, เลข, แชร์]"


In [ ]:
Corpus.to_excel('tweet-bullying-Corpus.xlsx', index = False, encoding = "utf-8-sig")

#STEP -5: Prepare Train and Test Data sets

In [ ]:
Corpus = pd.read_excel('tweet-bullying-Corpus.xlsx')

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['bully'],test_size=0.3)

In [ ]:
Train_X,Train_Y

(652     ['เป็นห่วง', 'ปี', 'แฟน', 'ได้ขนาด', 'อะ', 'ดู...
 549     ['อุ้ย', 'โดน', 'สลิ่ม', 'ด่าว่า', 'เปง', 'สวะ...
 713         ['คน', 'ดีๆ', 'เจอ', 'งี้', 'คำพูด', 'นิยาม']
 1064    ['รายการ', 'ทริป', 'กระบี่', 'อร์ค', 'กะเทย', ...
 644                                     ['จุ๊บ', 'เหม่ง']
                               ...                        
 1084    ['เพื่อน', 'อาย', 'ป่าว', 'อ่า', 'ใบ้ๆ', 'ชื่อ...
 697     ['เวลา', 'แนวนอน', 'มือ', 'ปิดหน้า', 'น่ารัก',...
 501      ['หน้า', 'ฝาน', 'เหมือน', 'เล่น', 'งิ้ว', 'อ่ะ']
 839               ['ทวิ', 'เอ๋อ', 'ปะ', 'ดู', 'โน', 'ติ']
 788                                             ['ถูกใจ']
 Name: text_final, Length: 840, dtype: object, 652      no
 549     yes
 713      no
 1064     no
 644      no
        ... 
 1084     no
 697      no
 501     yes
 839      no
 788      no
 Name: bully, Length: 840, dtype: object)

#STEP -6: Encoding

In [ ]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [ ]:
print(Train_Y, Test_Y)

[0 1 0 0 0 1 1 0 1 1 1 0 0 0 1 1 0 0 0 0 1 1 0 0 1 0 0 1 0 1 0 1 1 0 1 0 0
 1 1 0 0 0 0 0 1 1 1 1 0 0 1 0 0 1 1 0 0 1 0 0 1 1 1 0 1 1 0 0 1 0 1 1 1 0
 0 0 1 1 0 0 1 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 0 0 1 0 1 0 0
 1 1 1 1 0 1 1 1 1 1 0 1 0 1 0 1 1 1 1 0 0 0 0 0 1 0 0 1 1 0 1 1 1 1 1 1 0
 0 1 0 1 1 1 0 0 1 0 1 1 1 0 1 0 1 0 1 0 1 0 1 1 0 0 1 0 0 0 0 1 0 0 1 1 0
 0 0 1 0 0 0 1 1 1 0 1 1 1 1 0 1 1 0 0 1 1 0 1 0 1 1 0 0 1 0 1 0 1 0 0 0 0
 1 0 0 0 0 0 0 1 1 0 0 1 1 0 1 1 1 1 0 1 1 1 0 1 1 0 0 0 1 1 1 1 1 0 0 1 0
 1 0 0 0 0 1 1 0 0 1 0 1 1 0 1 1 1 0 1 0 0 0 1 1 0 0 0 1 0 1 1 1 1 0 1 1 0
 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 1 1 1 0 0 1 1 0 1 1 0 0 0 0 1 1 0 0 1 0 0 0
 1 1 0 0 0 0 1 1 1 0 0 0 0 1 0 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
 0 0 1 1 1 0 0 0 0 0 0 0 1 0 1 1 1 0 0 1 1 0 0 1 1 1 1 1 1 0 1 0 0 0 1 0 0
 0 0 1 1 1 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 1 0 0 0 0 0 1 1 1 0 1 1 1 1 0 0
 1 1 0 1 1 0 0 0 0 0 1 1 0 1 0 1 0 1 1 1 1 1 0 1 0 1 1 0 1 1 0 0 0 0 1 0 0
 0 0 0 1 0 1 0 1 1 1 0 0 

#STEP -7: Word Vectorization

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [ ]:
print(Tfidf_vect.vocabulary_)

{'คน': 184, 'เด': 1741, 'ยวน': 1109, 'ภาษา': 981, 'มารยาท': 1040, 'กร': 33, 'งเทพ': 342, 'เจอ': 1728, 'แบบน': 1936, 'าย': 1639, 'าน': 1621, 'สอน': 1352, 'เร': 1806, 'อง': 1466, 'แก': 1882, 'หาย': 1454, 'อาย': 1559, 'แฟน': 1953, 'ญญาอ': 439, 'อน': 1493, 'นๆ': 793, 'อย': 1518, 'เลย': 1814, 'เจ': 1722, 'เล': 1810, 'เง': 1718, 'ยน': 1081, 'กะเทย': 88, 'นกล': 676, 'าม': 1637, 'ผญ': 917, 'แต': 1908, 'งหน': 328, 'เหม': 1850, 'เก': 1692, 'นไป': 789, 'หน': 1407, 'ขาว': 172, 'ชมพ': 406, 'ตา': 583, 'ปาก': 904, 'แดง': 1904, 'ปร': 859, 'าบอ': 1630, 'ณหา': 459, 'ทางการ': 643, 'ดเพศ': 517, 'ขยะ': 140, 'ไอ': 2167, 'ดาก': 500, 'นแหละ': 782, 'สวะ': 1347, 'ขยะส': 141, 'งคม': 299, 'ขออภ': 163, 'หยาบ': 1427, 'ทน': 618, 'เดร': 1743, 'จฉาน': 364, 'เอดส': 1863, 'ตาย': 594, 'โลก': 2079, 'กท': 22, 'นน': 713, 'อแม': 1573, 'อให': 1577, 'เห': 1845, 'อนเด': 1505, 'แล': 1976, 'วก': 1226, 'ตอ': 569, 'พรากผ': 948, 'เยาว': 1805, 'เสน': 1836, 'ยดจ': 1076, 'ญไร': 447, 'วยซ': 1248, 'อนก': 1494, 'คคล': 179, 'ารำคาญ': 1646,

In [ ]:
print(Train_X_Tfidf)

  (0, 2151)	0.17275485082046202
  (0, 2135)	0.16502937948205995
  (0, 1976)	0.33415281013178993
  (0, 1953)	0.1780262539628739
  (0, 1820)	0.19367766835600275
  (0, 1810)	0.1216879509644066
  (0, 1773)	0.1491108910333739
  (0, 1745)	0.15042663738454365
  (0, 1639)	0.1578187400985245
  (0, 1548)	0.27702710365456745
  (0, 1466)	0.20952033401604844
  (0, 1228)	0.19367766835600275
  (0, 1226)	0.1876909615423995
  (0, 978)	0.25757513549577693
  (0, 741)	0.2387578613984549
  (0, 575)	0.1766360141958465
  (0, 478)	0.1982883007484897
  (0, 316)	0.25757513549577693
  (0, 213)	0.27251124621771444
  (0, 184)	0.10305532377794735
  (0, 134)	0.1554413357836661
  (0, 124)	0.24697779628968672
  (0, 0)	0.2320416855677492
  (1, 2035)	0.30642510760595626
  (1, 1774)	0.5029207077884001
  :	:
  (834, 768)	0.4465288438325958
  (834, 520)	0.46568853225190204
  (834, 20)	0.40925831098364873
  (835, 2122)	0.41813098414273225
  (835, 1785)	0.2747866878736255
  (835, 1647)	0.2843838379819705
  (835, 1570)	0.4302

#STEP -7: Use the ML Algorithms to Predict the outcome

In [ ]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  77.77777777777779


In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
matrix = classification_report(Test_Y,predictions_SVM, target_names=['Bully', 'notbully'], digits=4)
print(matrix)
print(confusion_matrix(Test_Y,predictions_SVM))
# from sklearn.metrics import plot_confusion_matrix
# # plot_confusion_matrix(Test_Y,predictions_SVM)  
# plt.show()

SVM Accuracy Score ->  82.77777777777777
              precision    recall  f1-score   support

       Bully     0.8148    0.8508    0.8324       181
    notbully     0.8421    0.8045    0.8229       179

    accuracy                         0.8278       360
   macro avg     0.8285    0.8276    0.8276       360
weighted avg     0.8284    0.8278    0.8277       360

[[154  27]
 [ 35 144]]
